In [251]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from functools import partial
import pandas as pd
from IPython.display import display
import ipywidgets as widgets

## Control of oscillations (OOP version)

In [252]:
# calc_others(sol, consts)
class oscillation:
    def __init__(self, model, params, consts, calc_all):

        # Model and parameters
        self.__model = model
        self.__params = params
        self.__consts = consts
        self.__calc_all = calc_all

        # Essential information
        self.__species = ['A2', 'S_total', 'A', 'O']
        self.__info = None

        # Experimental data
        self.__exp_data = None

    @property
    def info(self):
        print(
            f'The model includes {len(self.__params)} parameters and {len(self.__consts)} constants. The species are {self.__species}.')
        print(f'Additional information: {self.__info}')

    def add_info(self, info):
        self.__info = info

    def simulate(self, init_cond, t=10, t_eval='default'):
        params_pass = np.hstack((self.__params, self.__consts))
        model_partial = partial(self.__model, params=params_pass)
        t_span = (0, t)

        if type(t_eval) == str:
            t_eval = np.linspace(0, t, 500)
        sol = solve_ivp(model_partial, t_span=t_span,
                        y0=init_cond, t_eval=t_eval, rtol=1e-6, atol=1e-8)
        return sol

    def plot(self, init_cond, t=10, exp=False):
        i = 0
        if exp == True:
            sol = self.simulate(init_cond, t=self.__exp_data.iloc[-1, 0])
            c = self.__calc_all(sol, self.__consts)
            fig, axes = plt.subplots(3, 1, figsize=(8, 12))
            for ax in axes:
                ax.plot(self.__exp_data.iloc[:, 2*i], self.__exp_data.iloc[:, 2*i+1], label = 'exp')
                ax.plot(sol.t, c[i], label = self.__species[i])
                ax.legend()
                i += 1
        else:
            sol = self.simulate(init_cond, t)
            c = self.__calc_all(sol, self.__consts)
            fig, axes = plt.subplots(2, 2, figsize=(10, 6))
            for ax, y in zip(axes.flatten(), c):
                ax.plot(sol.t, y, label=self.__species[i])
                ax.legend()
                i += 1

        return fig, ax

    def interactive_plot(self, init_cond, t=10, range=5, step=0.05):

        if len(self.__params) == 4:
            print('4 parameters')

            def plot_temp(alpha, beta, theta, phi):
                params_old = self.__params
                params = [alpha, beta, theta, phi]
                self.__params = params
                self.plot(init_cond, t)
                self.__params = params_old

            alpha, beta, theta, phi = self.__params
            alpha_slider = widgets.FloatSlider(value=alpha, min=max(
                0, alpha-range), max=alpha+range, step=step, description='alpha')
            beta_slider = widgets.FloatSlider(value=beta, min=max(
                0, beta-range), max=beta+range, step=step, description='beta')
            theta_slider = widgets.FloatSlider(value=theta, min=max(
                0, theta-range), max=theta+range, step=step, description='theta')
            phi_slider = widgets.FloatSlider(value=phi, min=max(
                0, phi-range), max=phi+range, step=step, description='phi')

            interactive_widget = widgets.interactive(
                plot_temp, alpha=alpha_slider, beta=beta_slider, theta=theta_slider, phi=phi_slider)
            display(interactive_widget)

        elif len(self.__params) == 5:
            print('5 parameters')
            pass

    def add_exp_data(self, exp_data):
        print(
            f'The species are {self.__species}. Please check if the data is in the same order and correct format (time, concentration).')
        data = exp_data.clip(lower=0)
        self.__exp_data = data

    def set_params(self, params):
        self.__params = params

    def fit(self, plot=False):
        tA2 = np.array(self.__exp_data.iloc[:, 0])
        cA2 = np.array(self.__exp_data.iloc[:, 1])
        tS = np.array(self.__exp_data.iloc[:, 2])
        cS = np.array(self.__exp_data.iloc[:, 3])

        if self.__exp_data.shape[1] >= 4:
            tA = np.array(self.__exp_data.iloc[:, 4])
            cA = np.array(self.__exp_data.iloc[:, 5])

        t_span_A2 = tA2[-1]
        t_span_S = tS[-1]
        init_cond = [cA2[0], cS[0]]

        def objective(params):
            params_old = self.__params
            self.__params = params
            simA2 = self.simulate(init_cond, t=t_span_A2, t_eval=tA2)
            simS = self.simulate(init_cond, t=t_span_S, t_eval=tS)
            print(self.__params)
            c_all_A2 = self.__calc_all(simA2, self.__consts)
            c_all_S = self.__calc_all(simS, self.__consts)

            obj = np.sum((c_all_A2[0] - cA2)**2 + (c_all_S[1] - cS)**2)

            return obj

        opt_result = sp.optimize.minimize(
            objective, self.__params, method='Nelder-Mead', tol=1e-6, options={'disp': True})

        if plot == True:
            self.plot(init_cond, exp=True)
        return self.__params

In [253]:
#Params includes alpha, beta, theta, phi, lam, m in the final model

def model1(t, vars, params):
    alpha, beta, theta, phi, lam, m = params
    C_A2, C_S = vars
    dC_A2dt = 1 - alpha * C_A2 * C_S**m - theta * C_A2
    dC_Sdt = alpha/lam * C_A2 * C_S**m - beta * C_S**(m + 1) + theta/lam * C_A2 - phi * C_S
    return (dC_A2dt, dC_Sdt)

def model1_Hill(t, vars, params):
    pass

def calculate_all_conc(sol, consts):
    lam, m = consts
    C_A2 = sol.y[0]
    C_S = sol.y[1]
    C_M = C_S ** m
    C_A = 2 * (1 - C_A2) - lam * (C_S + C_M)
    C_O = 1 / (2 * (1 - C_A2) - lam * (C_S + C_M)) ** 2
    return (C_A2, C_S + C_M, C_A, C_O)

In [254]:
params = [15, 1, 8, 10]
consts = [0.17, 4]
init_cond = [0.1, 0]

model_test = oscillation(model1, params, consts, calculate_all_conc)

In [255]:
model_test.add_info('This is a test model. The original Fernando\'s model')
model_test.info

The model includes 4 parameters and 2 constants. The species are ['A2', 'S_total', 'A', 'O'].
Additional information: This is a test model. The original Fernando's model


In [256]:
df_c1 = pd.read_csv(r'Data_sets/1_expB_osc_NatChem_MH.csv')
df_c3b = pd.read_csv(r'Data_sets/3b_expB_osc_NatChem_MH.csv')
df_c4 = pd.read_csv(r'Data_sets/4_expB_osc_NatChem_MH.csv')
exp_data = pd.concat([df_c1, df_c3b, df_c4], axis=1)
exp_data.columns = ['t1', 'c1','t3b', 'c3b', 't4', 'c4']

cut_off = 95
CMC_3b = 0.52
C_A2tol = exp_data['c1'][0] + exp_data['c4'][0] / 2

exp_data = exp_data.head(cut_off)

exp_data['c1'] = exp_data['c1'] / C_A2tol
exp_data['c4'] = exp_data['c4'] / C_A2tol
exp_data['c3b'] = exp_data['c3b'] / CMC_3b

exp_data = exp_data.drop_duplicates(subset=['t1'])
exp_data = exp_data.drop_duplicates(subset=['t3b'])
exp_data = exp_data.drop_duplicates(subset=['t4'])
exp_data = exp_data.clip(lower=0)

model_test.add_exp_data(exp_data)

The species are ['A2', 'S_total', 'A', 'O']. Please check if the data is in the same order and correct format (time, concentration).


In [257]:
model_test.fit(plot=True)

[15.  1.  8. 10.]
[15.75  1.    8.   10.  ]
[15.    1.05  8.   10.  ]
[15.   1.   8.4 10. ]
[15.   1.   8.  10.5]
[15.375  1.025  8.2    9.5  ]
[15.28125  1.01875  8.15     9.75   ]
[15.09375  1.00625  8.05    10.25   ]
[15.234375  1.015625  8.125     9.875   ]
[15.4921875  0.9578125  8.2625     9.9375   ]
[15.73828125  0.98671875  8.39375     9.90625   ]
[16.10742188  0.98007813  8.590625    9.859375  ]
[15.16699219  0.97675781  8.6890625   9.8359375 ]
[14.87548828  0.96513672  9.03359375  9.75390625]
[15.50317383  0.93588867  9.01835938  9.90039062]
[15.63757324  0.89602051  9.46503906  9.91308594]
[15.3180542   0.96280518  9.48212891  9.82568359]
[15.9692688   0.90202026  9.88569336  9.67602539]
[14.79277039  0.88291321 10.34260254  9.72497559]
[14.13544464  0.83433075 11.21859131  9.65777588]
[15.65468216  0.83245163 10.99213257  9.78237915]
[16.0442791   0.76610909 11.97140198  9.7966156 ]
[14.59840679  0.8276125  11.18288727  9.92055511]
[14.88979769  0.69923124 12.4368309   9.81

KeyboardInterrupt: 